In [63]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn import preprocessing
import urllib
pd.set_option('display.max_columns', None)

In [304]:
curWeek = 9
year = 2020
gameThresh = 4

P5 = ['ACC','Pac-12','Big Ten','Big 12','SEC','Notre Dame']
G5 = ['Conference USA','Sun Belt','Mid-American','American Athletic','Mountain West','FBS Independents']

In [10]:
gamesDF = pd.read_json('https://api.collegefootballdata.com/games?year=%s&seasonType=regular'%(year))
gamesDF.head()

,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,401238035,2020,1,regular,2020-09-04T00:00:00.000Z,False,False,False,NaN,3803,Legion Field,5,UAB,Conference USA,45.0,"[14, 14, 3, 14]",0.998081,2110,Central Arkansas,None,35.0,"[7, 14, 0, 14]",0.001919,2.226002
1,401207101,2020,1,regular,2020-09-04T01:00:00.000Z,False,False,False,NaN,3815,M. M. Roberts Stadium,2572,Southern Mississippi,Conference USA,21.0,"[0, 10, 3, 8]",0.269739,6,South Alabama,Sun Belt,32.0,"[13, 0, 10, 9]",0.730261,5.586950
2,401237353,2020,1,regular,2020-09-05T17:00:00.000Z,False,False,False,NaN,3775,Joan C. Edwards Stadium,276,Marshall,Conference USA,59.0,"[21, 17, 14, 7]",0.998936,2198,Eastern Kentucky,None,0.0,"[0, 0, 0, 0]",0.001064,1.145603
3,401235700,2020,1,regular,2020-09-05T17:30:00.000Z,False,False,False,NaN,3841,Blaik Field at Michie Stadium,349,Army,FBS Independents,42.0,"[7, 14, 14, 7]",0.998940,2393,Middle Tennessee,Conference USA,0.0,"[0, 0, 0, 0]",0.001060,2.470913
4,401212484,2020,1,regular,2020-09-05T20:30:00.000Z,False,False,False,NaN,3644,Jim Wacker Field at Bobcat Stadium,326,Texas State,Sun Belt,24.0,"[0, 14, 7, 3]",0.046385,2567,SMU,American Athletic,31.0,"[0, 14, 14, 3]",0.953615,3.288221


In [14]:
gamesDF.loc[(gamesDF.away_team == 'Notre Dame'),'away_conference'] = 'Notre Dame'
gamesDF.loc[(gamesDF.home_team == 'Notre Dame'),'home_conference'] = 'Notre Dame'

In [19]:
teamsDF = pd.read_json('https://api.collegefootballdata.com/teams/fbs?year=%s'%(year))
FBSteams = np.array(teamsDF['school'])

In [465]:
teamsDF[teamsDF.school == 'Air Force']

,id,school,mascot,abbreviation,alt_name1,alt_name2,alt_name3,conference,division,color,alt_color,logos
0,2005,Air Force,Falcons,AFA,None,AFA,Air Force,Mountain West,Mountain,#004a7b,#ffffff,[http://a.espncdn.com/i/teamlogos/ncaa/500/200...


In [532]:
teamsDF.logos.iloc[0][0]

'http://a.espncdn.com/i/teamlogos/ncaa/500/2005.png'

In [30]:
FBSdict = {}
for team in FBSteams:
    FBSdict[team] = gamesDF[(gamesDF['away_team'] == team) | (gamesDF['home_team'] == team)]
    FBSdict[team] = FBSdict[team].reset_index()

In [31]:
FBSdict['Rutgers']

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,200,401247299,2020,8,regular,2020-10-24T16:00:00.000Z,False,False,True,NaN,3936,Spartan Stadium,127,Michigan State,Big Ten,27.0,"[7, 6, 7, 7]",0.679413,164,Rutgers,Big Ten,38.0,"[14, 14, 0, 10]",0.320587,5.958195
1,265,401247300,2020,9,regular,2020-10-31T19:30:00.000Z,False,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,21.0,"[7, 0, 8, 6]",0.003545,84,Indiana,Big Ten,37.0,"[3, 17, 10, 7]",0.996455,3.064368
2,302,401247310,2020,10,regular,2020-11-07T05:00:00.000Z,True,False,True,NaN,3861,Ohio Stadium,194,Ohio State,Big Ten,NaN,None,NaN,164,Rutgers,Big Ten,NaN,None,NaN,NaN
3,379,401247314,2020,11,regular,2020-11-14T05:00:00.000Z,True,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,NaN,None,NaN,356,Illinois,Big Ten,NaN,None,NaN,NaN
4,447,401247324,2020,12,regular,2020-11-21T05:00:00.000Z,True,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,NaN,None,NaN,130,Michigan,Big Ten,NaN,None,NaN,NaN
5,488,401247328,2020,13,regular,2020-11-28T05:00:00.000Z,True,False,True,NaN,3907,Ross-Ade Stadium,2509,Purdue,Big Ten,NaN,None,NaN,164,Rutgers,Big Ten,NaN,None,NaN,NaN
6,562,401247336,2020,14,regular,2020-12-05T05:00:00.000Z,True,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,NaN,None,NaN,213,Penn State,Big Ten,NaN,None,NaN,NaN
7,592,401247342,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,NaN,3665,Maryland Stadium,120,Maryland,Big Ten,NaN,None,NaN,164,Rutgers,Big Ten,NaN,None,NaN,NaN


In [37]:
FBSdict['Rutgers']

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,200,401247299,2020,8,regular,2020-10-24T16:00:00.000Z,False,False,True,NaN,3936,Spartan Stadium,127,Michigan State,Big Ten,27.0,"[7, 6, 7, 7]",0.679413,164,Rutgers,Big Ten,38.0,"[14, 14, 0, 10]",0.320587,5.958195
1,265,401247300,2020,9,regular,2020-10-31T19:30:00.000Z,False,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,21.0,"[7, 0, 8, 6]",0.003545,84,Indiana,Big Ten,37.0,"[3, 17, 10, 7]",0.996455,3.064368
2,302,401247310,2020,10,regular,2020-11-07T05:00:00.000Z,True,False,True,NaN,3861,Ohio Stadium,194,Ohio State,Big Ten,NaN,None,NaN,164,Rutgers,Big Ten,NaN,None,NaN,NaN
3,379,401247314,2020,11,regular,2020-11-14T05:00:00.000Z,True,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,NaN,None,NaN,356,Illinois,Big Ten,NaN,None,NaN,NaN
4,447,401247324,2020,12,regular,2020-11-21T05:00:00.000Z,True,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,NaN,None,NaN,130,Michigan,Big Ten,NaN,None,NaN,NaN
5,488,401247328,2020,13,regular,2020-11-28T05:00:00.000Z,True,False,True,NaN,3907,Ross-Ade Stadium,2509,Purdue,Big Ten,NaN,None,NaN,164,Rutgers,Big Ten,NaN,None,NaN,NaN
6,562,401247336,2020,14,regular,2020-12-05T05:00:00.000Z,True,False,True,NaN,6577,SHI Stadium,164,Rutgers,Big Ten,NaN,None,NaN,213,Penn State,Big Ten,NaN,None,NaN,NaN
7,592,401247342,2020,15,regular,2020-12-12T05:00:00.000Z,True,False,True,NaN,3665,Maryland Stadium,120,Maryland,Big Ten,NaN,None,NaN,164,Rutgers,Big Ten,NaN,None,NaN,NaN


In [69]:
def seasonProgression(row):
    return row.week/15

def winningTeam(row):
    if row['home_points'] > row['away_points']:
        return row['home_team']
    elif row['away_points'] > row['home_points']:
        return row['away_team']
    else:
        return None

def teamWin(row,team):
    if row.winner == team:
        return True
    elif row.winner == None:
        return None
    else:
        return False
    
def findOpp(row):
    if row.home == True:
        return row.away_team
    else:
        return row.home_team

def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference

def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.
    
def pointDiff(dfR,team):
    if dfR.home:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points
    
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15



In [77]:
FBSdict['Connecticut']

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index


In [85]:
FBSbar = tqdm(total=len(FBSteams))
for key in FBSdict:
    if len(FBSdict[key]) == 0:
        FBSbar.update(1)
        continue
    FBSbar.set_description(key)
    FBSdict[key]['seasonProg'] = FBSdict[key].apply(lambda row: seasonProgression(row),axis=1)
    FBSdict[key]['winner'] = FBSdict[key].apply(lambda row: winningTeam(row),axis=1)
    FBSdict[key]['teamWin'] = FBSdict[key].apply(lambda row: teamWin(row,key),axis=1)
    
    wins = 0
    losses = 0
    winList = []
    lossList = []
    
    for row in FBSdict[key].itertuples():
        
        if row.teamWin:
            wins += 1
        elif row.teamWin == None:
            wins += 0
        else:
            losses += 1
        winList.append(wins)
        lossList.append(losses)
    
    FBSdict[key]['wins'] = winList
    FBSdict[key]['losses'] = lossList
    
    hometeam = FBSdict[key]['home_team'] == key
    FBSdict[key]['home'] = hometeam
    
    FBSdict[key]['winPct'] = FBSdict[key]['wins']/(FBSdict[key]['wins'] + FBSdict[key]['losses'])
    FBSdict[key]['opp'] = FBSdict[key].apply(lambda row: findOpp(row),axis=1)
    FBSdict[key]['opp_conference'] = FBSdict[key].apply(lambda row: findOppConf(row),axis=1)
    FBSdict[key]['point_diff'] = FBSdict[key].apply(lambda row: pointDiff(row,key),axis=1)
    FBSdict[key]['loc_mult'] = FBSdict[key].apply(lambda row: locMult(row),axis=1)
    FBSdict[key]['conf_mult'] = FBSdict[key].apply(lambda row: confMult(row),axis=1)
    
    FBSbar.update(1)

In [103]:
FBSdict[FBSdict['Rutgers'].iloc[1].opp].loc[FBSdict[FBSdict['Rutgers'].iloc[1].opp].week == 9, 'winPct'].item()

1.0

In [224]:
def winQ(row,week):
    if row.week > week:
        return 0.
    
    if row.opp_conference == None:
        oppWinPct = 0.5
    else:
        opp = row.opp
        oppWinPct = FBSdict[opp].loc[FBSdict[opp].week <= week, 'winPct'].iloc[-1]
    
    if row.teamWin:
        winMult = 1
        oppFactor = oppWinPct
    else:
        winMult = -1
        oppFactor = 1 - oppWinPct
    
    return oppFactor * row.seasonProg * row.conf_mult * winMult*(row.loc_mult + 0.01*row.point_diff)
    

In [225]:
FBSdict['Air Force']

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index,seasonProg,winner,teamWin,wins,losses,home,winPct,opp,opp_conference,point_diff,loc_mult,conf_mult
0,117,401212557,2020,5,regular,2020-10-03T22:00:00.000Z,False,False,False,NaN,3713,Falcon Stadium,2005,Air Force,Mountain West,40.0,"[3, 13, 3, 21]",0.998374,2426,Navy,American Athletic,7.0,"[0, 7, 0, 0]",0.001626,4.705105,0.333333,Air Force,True,1,0,True,1.000000,Navy,American Athletic,33.0,1.1,0.5
1,239,401248993,2020,8,regular,2020-10-25T02:30:00.000Z,False,False,True,NaN,3935,Spartan Stadium,23,San José State,Mountain West,17.0,"[0, 0, 14, 3]",0.264795,2005,Air Force,Mountain West,6.0,"[0, 0, 0, 6]",0.735205,5.685858,0.533333,San José State,False,1,1,False,0.500000,San José State,Mountain West,-11.0,1.0,0.5
2,273,401248999,2020,9,regular,2020-10-31T22:00:00.000Z,False,False,True,NaN,3713,Falcon Stadium,2005,Air Force,Mountain West,30.0,"[14, 3, 0, 13]",0.192228,68,Boise State,Mountain West,49.0,"[7, 21, 7, 14]",0.807772,2.722715,0.600000,Boise State,False,1,2,True,0.333333,Boise State,Mountain West,-19.0,1.1,0.5
3,310,401207199,2020,10,regular,2020-11-07T16:30:00.000Z,False,False,False,NaN,3841,Blaik Field at Michie Stadium,349,Army,FBS Independents,NaN,None,NaN,2005,Air Force,Mountain West,NaN,None,NaN,NaN,0.666667,None,None,1,2,False,0.333333,Army,FBS Independents,NaN,1.0,0.5
4,403,401249011,2020,11,regular,2020-11-15T03:00:00.000Z,False,False,True,NaN,3984,War Memorial Stadium,2751,Wyoming,Mountain West,NaN,None,NaN,2005,Air Force,Mountain West,NaN,None,NaN,NaN,0.733333,None,None,1,2,False,0.333333,Wyoming,Mountain West,NaN,1.0,0.5
5,414,401249018,2020,12,regular,2020-11-21T02:30:00.000Z,False,False,True,NaN,3713,Falcon Stadium,2005,Air Force,Mountain West,NaN,None,NaN,167,New Mexico,Mountain West,NaN,None,NaN,NaN,0.800000,None,None,1,2,True,0.333333,New Mexico,Mountain West,NaN,1.1,0.5
6,466,401249023,2020,13,regular,2020-11-26T19:00:00.000Z,False,False,True,NaN,3713,Falcon Stadium,2005,Air Force,Mountain West,NaN,None,NaN,36,Colorado State,Mountain West,NaN,None,NaN,NaN,0.866667,None,None,1,2,True,0.333333,Colorado State,Mountain West,NaN,1.1,0.5
7,527,401249029,2020,14,regular,2020-12-04T02:30:00.000Z,False,False,True,NaN,3905,Romney Stadium,328,Utah State,Mountain West,NaN,None,NaN,2005,Air Force,Mountain West,NaN,None,NaN,NaN,0.933333,None,None,1,2,False,0.333333,Utah State,Mountain West,NaN,1.0,0.5


In [226]:
team = 'Air Force'
# winQ(FBSdict[team].iloc[0],7)
[winQ(FBSdict[team].iloc[0],i) for i in range(FBSdict[team].week.min(),curWeek+1)]

[0.07944444444444444,
 0.11916666666666667,
 0.14300000000000002,
 0.11916666666666667,
 0.10214285714285715]

In [227]:
FBSdict['San José State'].loc[FBSdict['San José State'].week <= curWeek]

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index,seasonProg,winner,teamWin,wins,losses,home,winPct,opp,opp_conference,point_diff,loc_mult,conf_mult
0,239,401248993,2020,8,regular,2020-10-25T02:30:00.000Z,False,False,True,NaN,3935,Spartan Stadium,23,San José State,Mountain West,17.0,"[0, 0, 14, 3]",0.264795,2005,Air Force,Mountain West,6.0,"[0, 0, 0, 6]",0.735205,5.685858,0.533333,San José State,True,1,0,True,1.0,Air Force,Mountain West,11.0,1.1,0.5
1,275,401249003,2020,9,regular,2020-10-31T23:00:00.000Z,False,False,True,NaN,3971,University Stadium,167,New Mexico,Mountain West,21.0,"[0, 14, 7, 0]",0.183969,23,San José State,Mountain West,38.0,"[7, 14, 10, 7]",0.816031,5.137373,0.600000,San José State,True,2,0,False,1.0,New Mexico,Mountain West,17.0,1.0,0.5


In [229]:
FBSbar2 = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar2.set_description(key)
    
    if len(FBSdict[key]) == 0:
        FBSbar2.update(1)
        continue
    
    for w in range(FBSdict[key].week.min(),curWeek+1):
        FBSdict[key]['win_q_%s'%w] = FBSdict[key].apply(lambda row: winQ(row,w),axis=1)
#         print(w)
        
    FBSbar2.update(1)

In [233]:
FBSdict['Syracuse']

,index,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index,seasonProg,winner,teamWin,wins,losses,home,winPct,opp,opp_conference,point_diff,loc_mult,conf_mult,win_q_2,win_q_3,win_q_4,win_q_5,win_q_6,win_q_7,win_q_8,win_q_9
0,12,401234565,2020,2,regular,2020-09-12T16:00:00.000Z,False,False,True,NaN,3787,Kenan Memorial Stadium,153,North Carolina,ACC,31.0,"[7, 0, 3, 21]",0.998956,183,Syracuse,ACC,6.0,"[0, 3, 3, 0]",0.001044,3.297854,0.133333,North Carolina,False,0,1,False,0.000000,North Carolina,ACC,-25.0,1.0,1.0,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.025000,-0.020000,-0.033333
1,38,401234571,2020,3,regular,2020-09-19T16:00:00.000Z,False,False,True,NaN,3752,Heinz Field,221,Pittsburgh,ACC,21.0,"[7, 7, 7, 0]",0.985418,183,Syracuse,ACC,10.0,"[3, 7, 0, 0]",0.014582,3.715639,0.200000,Pittsburgh,False,0,2,False,0.000000,Pittsburgh,ACC,-11.0,1.0,1.0,0.0,-0.0,-0.000000,-0.044500,-0.071200,-0.089000,-0.101714,-0.101714
2,59,401234594,2020,4,regular,2020-09-26T16:00:00.000Z,False,False,True,NaN,1964,Carrier Dome,183,Syracuse,ACC,37.0,"[17, 6, 7, 7]",0.850357,59,Georgia Tech,ACC,20.0,"[0, 13, 7, 0]",0.149643,7.295617,0.266667,Syracuse,True,1,2,True,0.333333,Georgia Tech,ACC,17.0,1.1,1.0,0.0,0.0,0.112889,0.112889,0.169333,0.135467,0.112889,0.096762
3,137,401234605,2020,6,regular,2020-10-10T16:30:00.000Z,False,False,True,NaN,1964,Carrier Dome,183,Syracuse,ACC,24.0,"[7, 7, 7, 3]",0.000491,150,Duke,ACC,38.0,"[10, 14, 6, 8]",0.999509,6.149375,0.400000,Duke,False,1,3,True,0.250000,Duke,ACC,-14.0,1.1,1.0,0.0,0.0,0.000000,0.000000,-0.307200,-0.320000,-0.320000,-0.274286
4,171,401234615,2020,7,regular,2020-10-17T16:00:00.000Z,False,False,False,NaN,1964,Carrier Dome,183,Syracuse,ACC,21.0,"[7, 7, 0, 7]",0.000477,2335,Liberty,FBS Independents,38.0,"[14, 7, 14, 3]",0.999523,3.265274,0.466667,Liberty,False,1,4,True,0.200000,Liberty,FBS Independents,-17.0,1.1,0.5,0.0,0.0,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000
5,201,401234618,2020,8,regular,2020-10-24T16:00:00.000Z,False,False,True,NaN,3836,Memorial Stadium,228,Clemson,ACC,47.0,"[17, 10, 7, 13]",0.978032,183,Syracuse,ACC,21.0,"[0, 14, 7, 0]",0.021968,1.191684,0.533333,Clemson,False,1,5,False,0.166667,Clemson,ACC,-26.0,1.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000
6,250,401234629,2020,9,regular,2020-10-31T16:00:00.000Z,False,False,True,NaN,1964,Carrier Dome,183,Syracuse,ACC,14.0,"[0, 7, 0, 7]",0.001104,154,Wake Forest,ACC,38.0,"[10, 7, 21, 0]",0.998896,2.120216,0.600000,Wake Forest,False,1,6,True,0.142857,Wake Forest,ACC,-24.0,1.1,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.172000
7,321,401234631,2020,10,regular,2020-11-07T19:00:00.000Z,False,False,True,NaN,1964,Carrier Dome,183,Syracuse,ACC,NaN,None,NaN,103,Boston College,ACC,NaN,None,NaN,NaN,0.666667,None,None,1,6,True,0.142857,Boston College,ACC,NaN,1.1,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,413,401234643,2020,12,regular,2020-11-21T00:00:00.000Z,False,False,True,NaN,3873,Papa John's Cardinal Stadium,97,Louisville,ACC,NaN,None,NaN,183,Syracuse,ACC,NaN,None,NaN,NaN,0.800000,None,None,1,6,False,0.142857,Louisville,ACC,NaN,1.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,501,401234655,2020,13,regular,2020-11-28T05:00:00.000Z,True,False,True,NaN,1964,Carrier Dome,183,Syracuse,ACC,NaN,None,NaN,152,NC State,ACC,NaN,None,NaN,NaN,0.866667,None,None,1,6,True,0.142857,NC State,ACC,NaN,1.1,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [234]:
def calcPct(team,week):
    winqweek = 'win_q_%s'%week
    wins = FBSdict[team].loc[FBSdict[team].week <= week,'wins'].iloc[-1]
    losses = FBSdict[team].loc[FBSdict[team].week <= week,'losses'].iloc[-1]
    return 0.9*FBSdict[team].loc[FBSdict[team].week <= week, winqweek].mean() + 0.01*(wins-losses)

In [250]:
calcPct('Air Force', 5)

0.08149999999999999

In [474]:
FBSbar3 = tqdm(total=len(FBSteams))
FBSpct = {}
for key in FBSdict:
    FBSbar3.set_description(key)
    if len(FBSdict[key]) == 0:
        FBSbar3.update(1)
        continue
    pctList = []
    for w in range(4,curWeek+1):
        if len(FBSdict[key][FBSdict[key].week <= w]) < gameThresh:
            pctList.append(None)
        else:
            try:
                pctList.append(calcPct(key,w))
            except IndexError:
                pctList.append(None)
    FBSpct[key] = pctList
    FBSbar3.update()
PCTdf = pd.DataFrame.from_dict(FBSpct,orient='index',columns=range(gameThresh,curWeek+1))
PCTdf.index = PCTdf.index.rename('School')
PCTdf.to_csv('%s/PCT.csv'%year)

In [475]:
PCTdf.sort_values(9,ascending=False).head(25)

,4,5,6,7,8,9
School,,,,,,
Alabama,NaN,NaN,NaN,0.267312,0.287534,0.297460
Clemson,NaN,NaN,0.180891,0.221909,0.215624,0.256295
Notre Dame,NaN,NaN,0.089870,0.123000,0.180683,0.198364
Cincinnati,NaN,NaN,NaN,0.090958,0.149467,0.187324
Georgia,NaN,NaN,NaN,0.167075,0.164930,0.185048
Coastal Carolina,NaN,NaN,NaN,0.116286,0.144734,0.168151
Texas A&M,NaN,NaN,NaN,0.117725,0.117725,0.162444
BYU,NaN,0.112267,0.147146,0.153662,0.155359,0.161789
Marshall,NaN,NaN,0.075135,0.108231,0.136421,0.148595


In [476]:
min_max_scaler = preprocessing.MinMaxScaler()
PCTnormdf = pd.DataFrame(min_max_scaler.fit_transform(PCTdf[PCTdf.columns]),index=PCTdf.index,columns=range(gameThresh,curWeek+1))

PCTnormdf.to_csv('%s/PCT_norm.csv'%year)

In [477]:
PCTnormdf.sort_values(9,ascending=False)

,4,5,6,7,8,9
School,,,,,,
Alabama,NaN,NaN,NaN,1.000000,1.000000,1.000000
Clemson,NaN,NaN,0.877063,0.892437,0.832477,0.916569
Notre Dame,NaN,NaN,0.577939,0.658118,0.751077,0.799157
Cincinnati,NaN,NaN,NaN,0.582211,0.678356,0.776781
Georgia,NaN,NaN,NaN,0.762534,0.714379,0.772169
...,...,...,...,...,...,...
Washington,NaN,NaN,NaN,NaN,NaN,NaN
Washington State,NaN,NaN,NaN,NaN,NaN,NaN
Western Michigan,NaN,NaN,NaN,NaN,NaN,NaN


In [478]:
Ranks = PCTdf.rank(method='first',ascending=False).fillna(999).astype('int64')
Ranks.to_csv('%s/Ranks.csv'%year)

In [479]:
Ranks.sort_values(9,ascending=True).head(25)

,4,5,6,7,8,9
School,,,,,,
Alabama,999,999,999,1,1,1
Clemson,999,999,2,2,2,2
Notre Dame,999,999,10,7,3,3
Cincinnati,999,999,999,17,8,4
Georgia,999,999,999,3,5,5
Coastal Carolina,999,999,999,10,9,6
Texas A&M,999,999,999,8,12,7
BYU,999,1,3,5,7,8
Marshall,999,999,12,13,10,9


In [480]:
Ranks.add_prefix('Rank_').loc['Alabama']

Rank_4    999
Rank_5    999
Rank_6    999
Rank_7      1
Rank_8      1
Rank_9      1
Name: Alabama, dtype: int64

In [481]:
PCTnormdf.add_prefix('PCT_').loc['Alabama']

PCT_4    NaN
PCT_5    NaN
PCT_6    NaN
PCT_7    1.0
PCT_8    1.0
PCT_9    1.0
Name: Alabama, dtype: float64

In [482]:
printDF = pd.concat([Ranks.add_prefix('Rank_W'),PCTnormdf.add_prefix('PCT_W')],join='inner',axis=1)

In [523]:
for w in range(4,curWeek+1):
    curWeekDF = printDF[['Rank_W%s'%w,'PCT_W%s'%w]].reset_index().dropna()
    curWeekDF['Team'] = curWeekDF.apply(lambda row: teamsDF[teamsDF.school == row['School']].abbreviation.item(),axis=1)
    curWeekDF.reset_index()[['Rank_W%s'%w,'School','PCT_W%s'%w]].sort_values('Rank_W%s'%w).to_csv('%s/W%s.csv'%(year,w),index=False)

In [521]:
curWeekDF = printDF[['Rank_W%s'%curWeek,'PCT_W%s'%curWeek]].reset_index().dropna()
curWeekDF['Team'] = curWeekDF.apply(lambda row: teamsDF[teamsDF.school == row['School']].abbreviation.item(),axis=1)
curWeekDF.reset_index()[['Rank_W%s'%curWeek,'School','PCT_W%s'%curWeek]].sort_values('Rank_W%s'%curWeek).to_csv('%s/W%s.csv'%(year,curWeek),index=False)

In [510]:
from tabulate import tabulate
print('WEEK %s RANKINGS\nRanking teams who have played at least %s games.\n'%(curWeek,gameThresh))
print(tabulate(curWeekDF.reset_index()[['Rank_W%s'%curWeek,'School','PCT_W%s'%curWeek]].sort_values('Rank_W9'),headers=['Rank','Team','PCT'],tablefmt='simple',showindex=False,floatfmt=".3f"))

WEEK 9 RANKINGS
Ranking teams who have played at least 4 games.

  Rank  Team                     PCT
------  ---------------------  -----
     1  Alabama                1.000
     2  Clemson                0.917
     3  Notre Dame             0.799
     4  Cincinnati             0.777
     5  Georgia                0.772
     6  Coastal Carolina       0.738
     7  Texas A&M              0.726
     8  BYU                    0.725
     9  Marshall               0.698
    10  Miami                  0.690
    11  Florida                0.671
    12  Wake Forest            0.649
    13  Oklahoma               0.632
    14  Oklahoma State         0.631
    15  Virginia Tech          0.619
    16  SMU                    0.619
    17  Louisiana              0.613
    18  Auburn                 0.597
    19  Texas                  0.584
    20  Florida Atlantic       0.583
    21  Liberty                0.579
    22  Army                   0.571
    23  Iowa State             0.567
    24  NC

In [516]:
print('WEEK %s RANKINGS\nPlayed at least %s games.\n'%(curWeek,gameThresh))
print(tabulate(curWeekDF.reset_index()[['Rank_W%s'%curWeek,'Team']].sort_values('Rank_W9').head(25),tablefmt='plain',showindex=False))

WEEK 9 RANKINGS
Played at least 4 games.

 1  ALA
 2  CLEM
 3  ND
 4  CIN
 5  UGA
 6  CCU
 7  TA&M
 8  BYU
 9  MRSH
10  MIAMI
11  FLA
12  WAKE
13  OKLA
14  OKST
15  VT
16  SMU
17  ULL
18  AUB
19  TEX
20  FAU
21  LIB
22  ARMY
23  ISU
24  NCST
25  UNC


,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,401238035,2020,1,regular,2020-09-04T00:00:00.000Z,False,False,False,NaN,3803,Legion Field,5,UAB,Conference USA,45.0,"[14, 14, 3, 14]",0.998081,2110,Central Arkansas,None,35.0,"[7, 14, 0, 14]",0.001919,2.226002
1,401207101,2020,1,regular,2020-09-04T01:00:00.000Z,False,False,False,NaN,3815,M. M. Roberts Stadium,2572,Southern Mississippi,Conference USA,21.0,"[0, 10, 3, 8]",0.269739,6,South Alabama,Sun Belt,32.0,"[13, 0, 10, 9]",0.730261,5.586950
2,401237353,2020,1,regular,2020-09-05T17:00:00.000Z,False,False,False,NaN,3775,Joan C. Edwards Stadium,276,Marshall,Conference USA,59.0,"[21, 17, 14, 7]",0.998936,2198,Eastern Kentucky,None,0.0,"[0, 0, 0, 0]",0.001064,1.145603
3,401235700,2020,1,regular,2020-09-05T17:30:00.000Z,False,False,False,NaN,3841,Blaik Field at Michie Stadium,349,Army,FBS Independents,42.0,"[7, 14, 14, 7]",0.998940,2393,Middle Tennessee,Conference USA,0.0,"[0, 0, 0, 0]",0.001060,2.470913
4,401212484,2020,1,regular,2020-09-05T20:30:00.000Z,False,False,False,NaN,3644,Jim Wacker Field at Bobcat Stadium,326,Texas State,Sun Belt,24.0,"[0, 14, 7, 3]",0.046385,2567,SMU,American Athletic,31.0,"[0, 14, 14, 3]",0.953615,3.288221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,401236016,2020,9,regular,2020-11-01T00:00:00.000Z,False,False,True,NaN,3784,Jones AT&T Stadium,2641,Texas Tech,Big 12,28.0,"[7, 7, 0, 14]",0.060292,201,Oklahoma,Big 12,62.0,"[21, 27, 7, 7]",0.939708,1.220792
283,401234628,2020,9,regular,2020-11-01T00:00:00.000Z,False,False,True,NaN,3923,Scott Stadium,258,Virginia,ACC,44.0,"[13, 14, 14, 3]",0.398995,153,North Carolina,ACC,41.0,"[10, 10, 7, 14]",0.601005,6.496427
284,401249004,2020,9,regular,2020-11-01T01:30:00.000Z,False,False,True,NaN,3905,Romney Stadium,328,Utah State,Mountain West,7.0,"[0, 7, 0, 0]",0.000903,21,San Diego State,Mountain West,38.0,"[7, 3, 14, 14]",0.999097,2.952686
285,401239887,2020,9,regular,2020-11-01T02:15:00.000Z,False,False,False,NaN,3801,LaVell Edwards Stadium,252,BYU,FBS Independents,41.0,"[14, 21, 3, 3]",0.990537,98,Western Kentucky,Conference USA,10.0,"[3, 0, 7, 0]",0.009463,1.165622
